In [4]:
import torch
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### Task 1

In [9]:
######TASK 1######
# code for downloading and formatting the data
transforms_fnc = transforms.Compose([
    transforms.Resize((784, 1)),
    transforms.ToTensor()
])

target_transform_fnc = transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), 1))

train_data = MNIST('./data', train=True, download=True, transform=transforms_fnc, target_transform=target_transform_fnc)
test_data = MNIST('./data', train=False, download=True, transform=transforms_fnc)

train_bs = len(train_data)
test_bs = len(test_data)

train_loader = iter(DataLoader(train_data, batch_size=train_bs, shuffle=False))
test_loader = iter(DataLoader(test_data, batch_size=test_bs, shuffle=False))

train_data_X, train_data_y = next(train_loader)
train_data_X = train_data_X.squeeze()

test_data_X, test_data_y = next(test_loader)
test_data_X = test_data_X.squeeze()

print('train data shape: {}, label shape: {}'.format(train_data_X.size(), train_data_y.size()))
print('test data shape: {}, label shape: {}'.format(test_data_X.size(), test_data_y.size()))

train data shape: torch.Size([60000, 784]), label shape: torch.Size([60000, 10])
test data shape: torch.Size([10000, 784]), label shape: torch.Size([10000])


### Task 2

In [ ]:
######TASK 2######
# code for minibatch SGD implementation

def _gradient():
    pass

def sgd_train(train_data_X_arg, train_data_y_arg, num_of_iterations, batch_size, learning_rate):
    # init weight
    weight = torch.empty(10, 784)
    torch.nn.init.zeros_(weight)

    uni_dist_weight = torch.ones(train_data_X_arg.size(0))
    for iter_idx in range(num_of_iterations):
        sampled_idx = torch.multinomial(uni_dist_weight, batch_size, replacement=True)

        pass

    pass